####Data Reading

In [0]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F
from pyspark.sql.window import Window


In [0]:
df=spark.sql("select * from databricks_catalog.silver.orders")
df.display()

In [0]:
df_dimcustomer=spark.sql("select Dimcustomerkey,Customer_id as dim_customer_id from databricks_catalog.gold.dimcustomer")
df_dimproducts=spark.sql("select product_id as Dimproductkey,product_id as dim_product_id from databricks_catalog.gold.dimproducts")
df_dimcustomer = df_dimcustomer.dropDuplicates(["dim_customer_id"])
df_dimproducts = df_dimproducts.dropDuplicates(["dim_product_id"])


####Fact Dataframe

In [0]:
df_fact=df.join(df_dimcustomer,df.customer_id==df_dimcustomer.dim_customer_id).join(df_dimproducts,df.product_id==df_dimproducts.dim_product_id)
df_fact_new=df_fact.drop("dim_customer_id","dim_product_id","customer_id","product_id")
df_fact_new.display()

In [0]:
#Check duplicates BEFORE merge
dup_df = (df_fact_new.groupBy("order_id", "Dimcustomerkey", "Dimproductkey").count().filter("count > 1"))

dup_df.show(50, False)


In [0]:
df_fact_new = df_fact_new.dropDuplicates(["order_id","Dimcustomerkey","Dimproductkey"])


####Upsert on fact table

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databricks_catalog.gold.FactOrders"):

    dlt_obj=DeltaTable.forName(spark,"databricks_catalog.gold.FactOrders")
    
    dlt_obj.alias("trg").merge(df_fact_new.alias("src"),"trg.order_id==src.order_id  and trg.Dimcustomerkey==src.Dimcustomerkey and trg.Dimproductkey==src.Dimproductkey" )\
        .whenMatchedUpdateAll()\
            .whenNotMatchedInsertAll().execute()
else:
    df_fact_new.write.format("delta").option("path","abfss://gold@databrickseteprojectsa.dfs.core.windows.net/FactOrders")\
        .saveAsTable("databricks_catalog.gold.FactOrders")

     

In [0]:
%sql
select * from databricks_catalog.gold.factorders